In [8]:
import numpy as np
import pandas as pd
import logging
from pathlib import Path
from fastlmm.association import single_snp
from pysnptools.util.mapreduce1.runner import Local, LocalMultiProc
from pysnptools.snpreader import Bed, Pheno, SnpData
from fastlmm.feature_selection.test import TestFeatureSelection

In [11]:
bedbase = '../../fastlmm/feature_selection/examples/toydata.5chrom.bed'
phen_fn = '../../fastlmm/feature_selection/examples/toydata.phe'
cov_fn = '../../fastlmm/feature_selection/examples/toydata.cov'
assert Path(phen_fn).exists()

tempout_dir = "tempout/single_snp"
os.makedirs(tempout_dir,exist_ok=True)

def file_name(testcase_name):
    temp_fn = os.path.join(tempout_dir,testcase_name+".txt")
    if os.path.exists(temp_fn):
        os.remove(temp_fn)
    return temp_fn


def compare_files(frame,ref_base):
    reffile = TestFeatureSelection.reference_file("single_snp/"+ref_base+".txt")
    assert Path(reffile).exists()

    #sid_list,pvalue_list = frame['SNP'].values,frame['Pvalue'].values

    #sid_to_pvalue = {}
    #for index, sid in enumerate(sid_list):
    #    sid_to_pvalue[sid] = pvalue_list[index]

    reference=pd.read_csv(reffile,delimiter='\s',comment=None,engine='python')
    assert len(frame) == len(reference), f"# of rows ({len(frame)} vs {len(reference)}) differs from file '{reffile}'"
    for _, row in reference.iterrows():
        sid = row.SNP
        pvalue = frame[frame['SNP'] == sid].iloc[0].PValue
        diff = abs(row.PValue - pvalue)
        if diff > 1e-5 or np.isnan(diff):
            raise Exception("pair {0} differs too much from file '{1}'".format(sid,reffile))
        assert abs(row.PValue - pvalue) < 1e-5, "wrong"


In [14]:
logging.info("TestSingleSnpLeaveOutOneChrom test_multipheno")
test_snps = Bed(bedbase, count_A1=False)
print(test_snps.shape)
pheno = phen_fn
covar = cov_fn

pheno2 = Pheno(pheno).read()
pheno2.val[0,0] = 100
pheno2.val[1,0] = -100

if True:
    pheno12 = SnpData(iid=pheno2.iid,sid=["pheno1","pheno2"],val = np.c_[Pheno(pheno).read().val,pheno2.val])
    output_file = file_name("multipheno12")
    frame = single_snp(test_snps[:,::10], pheno12,
                        force_full_rank=True,
                                covar=covar,
                                output_file_name=output_file,count_A1=False
                                )
    frame1 = frame[frame['Pheno']=='pheno1']
    del frame1['Pheno']
    compare_files(frame1,"two_looc")

    frame2 = frame[frame['Pheno']=='pheno2']
    del frame2['Pheno']
    compare_files(frame2,"multipheno2")


if True:
    pheno11 = SnpData(iid=pheno2.iid,sid=["pheno1a","pheno1b"],val = np.c_[Pheno(pheno).read().val,Pheno(pheno).read().val])
    output_file = file_name("multipheno11")
    frame = single_snp(test_snps[:,::10], pheno11,
                        force_full_rank=True,
                                covar=covar,
                                output_file_name=output_file,count_A1=False
                                )

    frame1 = frame[frame['Pheno']=='pheno1a']
    del frame1['Pheno']
    compare_files(frame1,"two_looc")

    frame2 = frame[frame['Pheno']=='pheno1b']
    del frame2['Pheno']
    compare_files(frame2,"two_looc")


if True:
    output_file = file_name("multipheno1")
    frame = single_snp(test_snps[:,::10], pheno,
                        force_full_rank=True,
                                covar=covar,
                                output_file_name=output_file,count_A1=False
                                )

    compare_files(frame,"two_looc")


if True:
    output_file = file_name("multipheno2")
    frame = single_snp(test_snps[:,::10], pheno2,
                        force_full_rank=True,
                                covar=covar,
                                output_file_name=output_file,count_A1=False
                                )

    compare_files(frame,"multipheno2")



if True:
    pheno22 = SnpData(iid=pheno2.iid,sid=["pheno2a","pheno2b"],val = np.c_[pheno2.val, pheno2.val])
    output_file = file_name("multipheno22")
    frame = single_snp(test_snps[:,::10], pheno22,
                        force_full_rank=True,
                                covar=covar,
                                output_file_name=output_file,count_A1=False
                                )
    frame1 = frame[frame['Pheno']=='pheno2a']
    del frame1['Pheno']
    compare_files(frame1,"multipheno2")

    frame2 = frame[frame['Pheno']=='pheno2b']
    del frame2['Pheno']
    compare_files(frame2,"multipheno2")

TestSingleSnpLeaveOutOneChrom test_multipheno
Loading fam file ..\..\fastlmm\feature_selection\examples\toydata.5chrom.fam
Loading bim file ..\..\fastlmm\feature_selection\examples\toydata.5chrom.bim
(500, 10000)
Setting GB_goal to 0.1446951289176941 GB
Starting '_read_with_standardizing'
reading 715 SNPs in blocks of 500 and adding up kernels (for 500 individuals) with numpy.
0.02 seconds elapsed
Ending '_read_with_standardizing'
Starting findH2
h2=[0.14177116]
Starting '_read_with_standardizing'
reading 715 SNPs in blocks of 500 and adding up kernels (for 500 individuals) with numpy.
0.02 seconds elapsed
Ending '_read_with_standardizing'
Starting findH2
h2=[0.]
cmk
time=0.04199719429016113
Setting GB_goal to 0.1446951289176941 GB
Starting '_read_with_standardizing'
reading 638 SNPs in blocks of 500 and adding up kernels (for 500 individuals) with numpy.
0.02 seconds elapsed
Ending '_read_with_standardizing'
Starting findH2
h2=[0.03043305]
Starting '_read_with_standardizing'
reading 6

In [22]:
def pheno_dup(y_count):
    pheno0 = Pheno(phen_fn).read()
    iid = pheno0.iid
    sid = [f"sid{0}" for sid_index in range(y_count)]
    val = np.repeat(pheno0.val,y_count,axis=1)
    snpdata = SnpData(iid=iid,sid=sid,val=val,name=f"pheno_dup({y_count})")
    return snpdata

pheno_dup(3)    

SnpData(pheno_dup(3))

In [30]:
%%time
y_count = 1
runner = LocalMultiProc(10)

frame = single_snp(test_snps[:,::1], pheno_dup(y_count),
                    force_full_rank=True,
                            covar=covar,
                            runner=runner
                            )

# compare_files(frame,"two_looc")


PhenotypeName	sid0
SampleSize	500
SNPCount	10000
Runtime	7.224602699279785
Wall time: 7.24 s
